In [1]:
#  Dependencies
import requests
from elasticsearch import Elasticsearch,helpers
import numpy as np
import uuid
import random
import json

In [2]:
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],
                                 timeout=10000)

In [3]:
es.ping()

True

In [4]:
print(es.info(), '\n', es.cluster.health())

{'cluster_name': 'atlas', 'cluster_uuid': 'HvlJZuzTRReL9Gq1KVSIHQ', 'version': {'build_hash': '260387d', 'build_snapshot': False, 'number': '5.5.0', 'lucene_version': '6.6.0', 'build_date': '2017-06-30T23:16:05.735Z'}, 'tagline': 'You Know, for Search', 'name': 'es-head02-master'} 
 {'active_shards_percent_as_number': 100.0, 'active_primary_shards': 12629, 'number_of_nodes': 10, 'status': 'green', 'delayed_unassigned_shards': 0, 'initializing_shards': 0, 'relocating_shards': 0, 'timed_out': False, 'unassigned_shards': 0, 'task_max_waiting_in_queue_millis': 0, 'number_of_in_flight_fetch': 0, 'number_of_pending_tasks': 0, 'cluster_name': 'atlas', 'number_of_data_nodes': 5, 'active_shards': 17589}


In [5]:
# checking indices

indices = es.indices.get_aliases().keys()
print('Total No. of Indices: ',len(indices),'\n')
# print('\n', indices)
rucio=(index for index in indices if('rucio-events' in index))
network_weather = (index for index in indices if('network_weather-2017' in index))
rucio_indices = []
nws_indices = []
for event in network_weather:
    nws_indices.append(event)
for event in rucio:
    rucio_indices.append(event)
print('total NWS indices:',len(nws_indices),'\n')
print(nws_indices[0:5], '\n')

print('total rucio indices:',len(rucio_indices),'\n')
print(rucio_indices[0:5])

Total No. of Indices:  2596 

total NWS indices: 227 

['network_weather-2017.4.4', 'network_weather-2017.5.31', 'network_weather-2017.2.8', 'network_weather-2017.1.1', 'network_weather-2017.1.9'] 

total rucio indices: 527 

['rucio-events-2016.07.12', 'rucio-events-2016.04.27', 'rucio-events-2016.10.02', 'rucio-events-2017.06.25', 'rucio-events-2017.03.27']


In [6]:
count=es.count(index='network_weather-2017*')
print('total documents : {}'.format( count['count']) )

total documents : 1442654847


In [7]:
nws_indices_dict = {}
for event in nws_indices:
    i = es.count(index=event)
    nws_indices_dict[event] = i['count']
# print('total data points:',sum(int(list(indices_dict.values()))))

print(nws_indices_dict)

{'network_weather-2017.7.23': 16976379, 'network_weather-2017.6.15': 16272220, 'network_weather-2017.4.18': 2940298, 'network_weather-2017.8.4': 15982071, 'network_weather-2017.1.9': 3957564, 'network_weather-2017.7.6': 16902500, 'network_weather-2017.4.17': 3267682, 'network_weather-2017.1.3': 4617257, 'network_weather-2017.6.25': 14720576, 'network_weather-2017.2.20': 4004030, 'network_weather-2017.3.20': 3525786, 'network_weather-2017.8.12': 16588484, 'network_weather-2017.7.22': 16415312, 'network_weather-2017.5.12': 14624637, 'network_weather-2017.5.7': 14426727, 'network_weather-2017.1.10': 3897760, 'network_weather-2017.5.31': 16780747, 'network_weather-2017.3.30': 4259944, 'network_weather-2017.7.27': 16647094, 'network_weather-2017.1.29': 3424128, 'network_weather-2017.3.19': 3504319, 'network_weather-2017.3.5': 4203370, 'network_weather-2017.2.13': 3953980, 'network_weather-2017.7.11': 16402928, 'network_weather-2017.1.16': 3661202, 'network_weather-2017.1.1': 4379687, 'netwo

In [11]:
def extract_data(index, query, scan_size, scan_step):
    resp = es.search(
    index = index,
    scroll = '20m',
    body = query,
    size = scan_step)

    sid = resp['_scroll_id']
    scroll_size = resp['hits']['total']
    print('total hits in {} : {}'.format(index,scroll_size))
    results=[]
    for hit in resp['hits']['hits']:
        results.append(hit)
    #plot_data_stats(results)
    steps = int((scan_size-scan_step)/ scan_step)

    # Start scrolling

    for i in range(steps):
        if i%10==0:
            print("Scrolling index : {} ; step : {} ...\n ".format(index,i))
        resp = es.scroll(scroll_id = sid, scroll = '20m')
        # Update the scroll ID
        sid = resp['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(resp['hits']['hits'])
        if i%10==0:
            print("scroll size: " + str(scroll_size))
        for hit in resp['hits']['hits']:
            results.append(hit)
    
    print("\n Done Scrolling through {} !! \n".format(index))
    results = pd.DataFrame(results)
    print(results.info(), '\n')
    return results

In [12]:
import numpy as np
import pandas as pd
nws= extract_data(index='network_weather-2017.8.10',query={}, scan_size=1000000, scan_step=10000)

total hits in network_weather-2017.8.10 : 16231686
Scrolling index : network_weather-2017.8.10 ; step : 0 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 10 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 20 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 30 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 40 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 50 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 60 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 70 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 80 ...
 
scroll size: 10000
Scrolling index : network_weather-2017.8.10 ; step : 90 ...
 
scroll size: 10000

 Done Scrolling through network_weather-2017.8.10 !! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data 

In [13]:
nws.to_csv('nws.csv')

In [16]:
nws['_type'].unique()

array(['latency', 'packet_loss_rate', 'ps_perf', 'link_utilization',
       'traceroute', 'throughput', 'netflow_lhcopn', 'retransmits'], dtype=object)

In [17]:
nws['_type'].value_counts()

packet_loss_rate    457196
latency             455104
link_utilization     56595
traceroute           26810
netflow_lhcopn        1567
throughput            1070
ps_perf                839
retransmits            819
Name: _type, dtype: int64

In [21]:
myquery = {
    "query": {
        "term": {
            '_type': 'throughput'
            }
        }
    }
es.search(index='network_weather-2017.8.10',body={}, size = 1)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AV3JkuUXNILXDji87ilt',
    '_index': 'network_weather-2017.8.10',
    '_score': 1.0,
    '_source': {'MA': '128.227.221.44',
     'delay_mean': 63.81549999999999,
     'delay_median': 63.8,
     'delay_sd': 0.08856118412336961,
     'dest': '148.6.8.251',
     'destProduction': True,
     'destSite': 'BUDAPEST',
     'destVO': 'ATLAS',
     'dest_host': 'psbud01.kfki.hu',
     'ipv6': False,
     'src': '128.227.221.44',
     'srcProduction': True,
     'srcSite': 'UFlorida-HPC',
     'srcVO': '',
     'src_host': 'perfsonar1.ihepa.ufl.edu',
     'timestamp': 1502325310000},
    '_type': 'latency'}],
  'max_score': 1.0,
  'total': 16231686},
 'timed_out': False,
 'took': 41}

In [38]:
query2={
    "query": {
        "term": {
            '_type': 'throughput'
            }
        }
    }
import pandas as pd
throughput= extract_data(index='network_weather-2017.8.10', query=query2, scan_size=15033, scan_step=10000)

total hits in network_weather-2017.8.10 : 15033

 Done Scrolling through network_weather-2017.8.10 !! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
MA                10000 non-null object
dest              10000 non-null object
destProduction    10000 non-null bool
destSite          6958 non-null object
destVO            6958 non-null object
dest_host         10000 non-null object
ipv6              10000 non-null bool
src               10000 non-null object
srcProduction     10000 non-null bool
srcSite           6886 non-null object
srcVO             6886 non-null object
src_host          10000 non-null object
throughput        10000 non-null float64
timestamp         10000 non-null int64
dtypes: bool(3), float64(1), int64(1), object(9)
memory usage: 888.8+ KB
None 



In [39]:
throughput.head()

,MA,dest,destProduction,destSite,destVO,dest_host,ipv6,src,srcProduction,srcSite,srcVO,src_host,throughput,timestamp
0,137.222.74.17,137.222.74.17,True,UKI-SOUTHGRID-BRIS-HEP,ATLAS,lcgnetmon02.phy.bris.ac.uk,False,169.228.130.41,True,NaN,NaN,perfsonar-1.t2.ucsd.edu,1.866498e+08,1502324812000
1,165.91.55.6,192.12.15.111,True,BNL-ATLAS,ATLAS,ps-development.bnl.gov,False,165.91.55.6,True,NaN,NaN,psonar-bwctl.brazos.tamu.edu,9.416140e+08,1502324895000
2,18.12.1.172,18.12.1.172,True,NaN,NaN,18.12.1.172,False,198.124.238.66,False,NaN,NaN,bost-pt1.es.net,7.821352e+09,1502324969000
3,192.111.108.111,129.93.239.163,True,Nebraska,,hcc-ps02.unl.edu,False,192.111.108.111,True,Vanderbilt,ATLAS,192.111.108.111,2.270000e+08,1502324971000
4,129.215.213.69,129.215.213.69,True,UKI-SCOTGRID-ECDF,ATLAS,gridpp-ps-band.ecdf.ed.ac.uk,False,195.194.105.177,True,UKI-NORTHGRID-MAN-HEP,ATLAS,perfsonar-bw.tier2.hep.manchester.ac.uk,3.628617e+09,1502325054000


In [73]:
cond1 = throughput['destSite'] ==float('nan')
a= throughput[not(cond1)]
# cond2 = a['srcSite'] !=float('nan')
# a= a[cond2]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [70]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 14 columns):
MA                10000 non-null object
dest              10000 non-null object
destProduction    10000 non-null bool
destSite          6958 non-null object
destVO            6958 non-null object
dest_host         10000 non-null object
ipv6              10000 non-null bool
src               10000 non-null object
srcProduction     10000 non-null bool
srcSite           6886 non-null object
srcVO             6886 non-null object
src_host          10000 non-null object
throughput        10000 non-null float64
timestamp         10000 non-null int64
dtypes: bool(3), float64(1), int64(1), object(9)
memory usage: 966.8+ KB


In [71]:
a.head()
# type(a['destSite'][2])

,MA,dest,destProduction,destSite,destVO,dest_host,ipv6,src,srcProduction,srcSite,srcVO,src_host,throughput,timestamp
0,137.222.74.17,137.222.74.17,True,UKI-SOUTHGRID-BRIS-HEP,ATLAS,lcgnetmon02.phy.bris.ac.uk,False,169.228.130.41,True,NaN,NaN,perfsonar-1.t2.ucsd.edu,1.866498e+08,1502324812000
1,165.91.55.6,192.12.15.111,True,BNL-ATLAS,ATLAS,ps-development.bnl.gov,False,165.91.55.6,True,NaN,NaN,psonar-bwctl.brazos.tamu.edu,9.416140e+08,1502324895000
2,18.12.1.172,18.12.1.172,True,NaN,NaN,18.12.1.172,False,198.124.238.66,False,NaN,NaN,bost-pt1.es.net,7.821352e+09,1502324969000
3,192.111.108.111,129.93.239.163,True,Nebraska,,hcc-ps02.unl.edu,False,192.111.108.111,True,Vanderbilt,ATLAS,192.111.108.111,2.270000e+08,1502324971000
4,129.215.213.69,129.215.213.69,True,UKI-SCOTGRID-ECDF,ATLAS,gridpp-ps-band.ecdf.ed.ac.uk,False,195.194.105.177,True,UKI-NORTHGRID-MAN-HEP,ATLAS,perfsonar-bw.tier2.hep.manchester.ac.uk,3.628617e+09,1502325054000


# INDEX- network-weather-2017*

## event format - 

```json
{
    "_index": "jobs_archive_2017-06-05",
    "_type": "jobs_data",
    "_id": "3413570266",
    "_version": 1,
    "_score": null,
    "_source": {
        "pandaid": 3413570266,
        "jobdefinitionid": 146514542,
        "schedulerid": "aCT-atlact1-ai403",
        "pilotid": "http://aipanda403.cern.ch/data/jobs/20170605/jost.arnes.si/YWRKDmXP0bqnmmR0Xox1SiGmABFKDmABFKDmSMIKDmGBGKDmUdp8Lo|NEWMOVER-ON|SLURM|PR|PICARD 68.3",
        "creationtime": "2017-06-05T06:02:41",
        "creationhost": "hammercloud-ai-74.ipv6.cern.ch",
        "modificationtime": "2017-06-05T06:59:50",
        "modificationhost": "wn095.arnes.si",
        "atlasrelease": "Atlas-20.7.6",
        "transformation": "http://pandaserver.cern.ch:25080/trf/user/runAthena-00-00-11",
        "homepackage": "AnalysisTransforms-AtlasDerivation_20.7.6.4",
        "prodserieslabel": null,
        "prodsourcelabel": "user",
        "produserid": "/DC=ch/DC=cern/OU=Organic Units/OU=Users/CN=gangarbt/CN=601592/CN=Robot: Ganga Robot/CN=proxy",
        "assignedpriority": 0,
        "currentpriority": 4000,
        "attemptnr": 0,
        "maxattempt": 2,
        "jobstatus": "finished",
        "jobname": "90084d6f-22ef-462f-8fb0-b383e7e4629b_67487",
        "maxcpucount": 0,
        "maxdiskcount": 0,
        "minramcount": 0,
        "starttime": "2017-06-05T06:29:31",
        "endtime": "2017-06-05T06:49:49",
        "cpuconsumptiontime": 834,
        "cpuconsumptionunit": "s+Intel(R) Xeon(R) CPU E5-2650 v2 @ 2.60GHz 20480 KB",
        "commandtopilot": null,
        "transexitcode": null,
        "piloterrorcode": 0,
        "piloterrordiag": null,
        "exeerrorcode": 0,
        "exeerrordiag": "OK",
        "superrorcode": 0,
        "superrordiag": null,
        "ddmerrorcode": 0,
        "ddmerrordiag": null,
        "brokerageerrorcode": 0,
        "brokerageerrordiag": null,
        "jobdispatchererrorcode": 0,
        "jobdispatchererrordiag": null,
        "taskbuffererrorcode": 0,
        "taskbuffererrordiag": null,
        "computingsite": "ANALY_ARNES_DIRECT",
        "computingelement": "jost.arnes.si",
        "proddblock": "data15_13TeV.00276336.physics_Main.merge.AOD.r7562_p2521_tid07709524_00",
        "dispatchdblock": null,
        "destinationdblock": "user.gangarbt.hc20107254.tid845.ANALY_ARNES_DIRECT.6",
        "destinationse": "ANALY_ARNES_DIRECT",
        "nevents": 3903,
        "grid": null,
        "cloud": "ND",
        "cpuconversion": "1",
        "sourcesite": null,
        "destinationsite": null,
        "transfertype": null,
        "taskid": 2,
        "cmtconfig": "x86_64-slc6-gcc49-opt",
        "statechangetime": "2017-06-05T06:59:50",
        "lockedby": null,
        "relocationflag": 1,
        "jobexecutionid": 0,
        "vo": "atlas",
        "workinggroup": null,
        "processingtype": "gangarobot",
        "produsername": "gangarbt",
        "countrygroup": null,
        "batchid": "12405832",
        "parentid": null,
        "specialhandling": null,
        "jobsetid": null,
        "corecount": 1,
        "ninputdatafiles": 1,
        "inputfiletype": "AOD",
        "inputfileproject": "data15_13TeV",
        "inputfilebytes": 824476423,
        "noutputdatafiles": 1,
        "outputfilebytes": 4025868,
        "dbTime": 29.64,
        "dbData": 8090396,
        "workDirSize": 1650688,
        "jobmetrics": null,
        "workqueue_id": null,
        "jeditaskid": null,
        "jobsubstatus": null,
        "actualcorecount": 1,
        "reqid": null,
        "maxrss": 1483408,
        "maxvmem": 3293680,
        "maxpss": 1411801,
        "avgrss": 1102786,
        "avgvmem": 2744469,
        "avgswap": 6041,
        "avgpss": 1042013,
        "maxwalltime": null,
        "wall_time": 1218,
        "cpu_eff": 0.68472904,
        "queue_time": 1610,
        "timeGetJob": 0,
        "timeStageIn": 17,
        "timeExe": 1038,
        "timeStageOut": 15,
        "timeSetup": 0,
        "nucleus": null,
        "eventservice": null,
        "failedattempt": null,
        "hs06sec": null,
        "hs06": null,
        "gShare": "Analysis",
        "IOcharRead": 753904,
        "IOcharWritten": 17778,
        "IObytesRead": 3151948,
        "IObytesWritten": 23136,
        "IOcharReadRate": 766631,
        "IOcharWriteRate": 18078,
        "IObytesReadRate": 3205158,
        "IObytesWriteRate": 23526
    },
    "fields": {
        "Average RSS": [
            1129252864
        ],
        "Average PSS": [
            1067021312
        ],
        "IObytesWritten_Bytes": [
            23691264
        ],
        "modificationtime": [
            1496645990000
        ],
        "wall_time_cores_h": [
            0.3383333333333333
        ],
        "Max_PSS_per_core": [
            1445684224
        ],
        "starttime": [
            1496644171000
        ],
        "TotalJobTime": [
            2828
        ],
        "IOcharRead_Bytes": [
            771997696
        ],
        "dbRate": [
            0.26031049371244186
        ],
        "Wall time per event times core": [
            0.31206764027671025
        ],
        "trf_scripted": [
            "runAthena-00-00-11"
        ],
        "Average VMEM": [
            2810336256
        ],
        "cpueff_per_core_painless": [
            0.6847290640394089
        ],
        "Max VMEM": [
            3372728320
        ],
        "infile_proj": [
            "data15_13TeV"
        ],
        "CPU eff per core * 100": [
            68.47290396690369
        ],
        "Max PSS": [
            1445684224
        ],
        "statechangetime": [
            1496645990000
        ],
        "Max RSS": [
            1519009792
        ],
        "endtime": [
            1496645389000
        ],
        "temp_timeOther": [
            148
        ],
        "CPU eff per core": [
            0.6847290396690369
        ],
        "sharedmem": [
            71607
        ],
        "CPU eff of 40 core jobs": [
            0.01711822599172592
        ],
        "CPU eff of 8 core jobs": [
            0.08559112995862961
        ],
        "IOcharWritten_Bytes": [
            18204672
        ],
        "Wall time per event * 10": [
            3.1206764027671023
        ],
        "cputimeperevent": [
            0.21368178324365872
        ],
        "IObytesRead_Bytes": [
            3227594752
        ],
        "cpueff_per_core_over_timeExe": [
            80.34682080924856
        ],
        "inputRate": [
            48498613.11764706
        ],
        "cputimeperevent seconds": [
            0.21368178324365872
        ],
        "cpu_eff_wen": [
            0.6847290640394089
        ],
        "Walltime times core": [
            1218
        ],
        "creationtime": [
            1496642561000
        ],
        "Wall time per event": [
            0.31206764027671025
        ],
        "top_country": [
            null
        ],
        "CPU eff of 6 core jobs": [
            11.41214609629035
        ],
        "timeExe_Core_Events": [
            0.26594926979246736
        ]
    },
    "sort": [
        1496645990000
    ]
}
```